In [1]:
# ------------------------------------------------------------
# 1. IMPORTS
# ------------------------------------------------------------
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.20.0


In [2]:
# ------------------------------------------------------------
# 2. DATASET PATHS
# ------------------------------------------------------------
TRAIN_DIR = "data/train"        # MUST contain "open" and "closed" subfolders
VAL_DIR   = "data/val"

IMG_SIZE = (64, 64)
BATCH = 32
COLOR_MODE = "grayscale"   # or "rgb"


In [ ]:
# ------------------------------------------------------------
# 3. DATA GENERATORS
# ------------------------------------------------------------
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=8,
    width_shift_range=0.08,
    height_shift_range=0.08,
    shear_range=0.05,
    zoom_range=0.10,
).flow_from_directory(
    TRAIN_DIR,
    class_mode="binary",
    color_mode=COLOR_MODE,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=True
)

val_gen = ImageDataGenerator(
    rescale=1./255,
).flow_from_directory(
    VAL_DIR,
    class_mode="binary",
    color_mode=COLOR_MODE,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False
)


In [ ]:
# ------------------------------------------------------------
# 4. MODEL ARCHITECTURE (SMALL CNN)
# ------------------------------------------------------------
input_shape = IMG_SIZE + (1,)  # grayscale

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
# ------------------------------------------------------------
# 5. TRAINING
# ------------------------------------------------------------
EPOCHS = 15

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)


In [ ]:
# ------------------------------------------------------------
# 6. PLOT TRAINING CURVES
# ------------------------------------------------------------
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title("Loss")
plt.legend()

plt.show()


In [ ]:
# ------------------------------------------------------------
# 7. SAVE MODEL (.h5)
# ------------------------------------------------------------
model.save("eye_cnn.h5")
print("Model saved as eye_cnn.h5")


In [ ]:
# ------------------------------------------------------------
# 8. EXPORT TFLITE MODEL
# ------------------------------------------------------------
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # smaller, faster
tflite_model = converter.convert()

with open("eye_cnn.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as eye_cnn.tflite")
